# Monoids

[Bartosz Milewsky](https://bartoszmilewski.com/) has a great series of articles on category theory as it applies to programmers.
I'm not going to lie, the material is pretty dense which is par for the course for a lot of mathematics subjects (maybe it's worth mentioning that Bartosz has a PhD in Theoretical Physics 😬).
However, many of the core concepts are reasonably simple abstractions, and programmers know all about abstractions!
The language can be as big a hurdle as reading mathematics symbols, but these are the tools mathematicians use to convey ideas in an elegant and concise way.
Hopefully, with a bit of patience, we can uncover the power of these ideas and see how they apply to our day-to-day jobs!

**Disclaimer** This is not meant to be a mathematically rigorous discussion.  Many small details are overlooked in favor of a high-level understanding.

## Definition

First, I suppose we should start with a definition - what _is_ a monoid?
According to Eric Weisstein on Wolfram, a monoid is "a set that is closed under an associative binary operation and has an identity element $I \in S$ such that $ \forall a \in S, Ia=aI=a$".
The set $S$ is often called the "carrier set".

See, it's easy! 🎉

Stay with me, sarcasm aside, it's not that bad!
I promise that you know what monoids are - you use them every day as a programmer!
Let's deconstruct that phrase a bit.
As a programmer, you _should_ know what a set is - an unordered collection of unique items of the same type.
The integers form a set, $\mathbb{I} = \{-1,0,1,2,3,4...\}$.
Rational numbers form a set, $\mathbb{Q} = \{-99.42,0,1.1,1.2,1.3...\}$.
For the purposes of this article, a "set" is a mathematical set, not constrained by physical memory limits of computers.

The next part, "closed under an associative binary operation" has three parts.

- **Closed** means that the operation on an object in our set will keep us in our set.  For example, the minimum of two integers maps back into the set of integers.  Multiplication of two integers stays within the set.  Subtraction and the set of *natural numbers* does **not** always stay within the natural numbers, this it is not closed (1 - 3 dumps you into negative integers).
- **Binary operation** means an operation that takes two values from the set; addition, multiplication, division, etc.  Most of the Python numeric [dunder methods](https://docs.python.org/3/reference/datamodel.html#emulating-numeric-types) are binary operations.
- **Associative** means that order doesn't matter.  1 + (2 + 3) is the same as (1 + 2) + 3.  Subtraction is not associative as order matters.  1 - (2 - 3) is **not equal** to (1 - 2) - 3.

We're almost there!
The last part really is easy.
It says that there is an identity element in the set such that the binary operation performed with the identity and some $a$ in the set returns $a$.
You already know what a few of these are!
- Numeric addition with 0 as the identity.
- Numeric multiplication with 1 as the identity (gotcha!  The identity is dependent on the _operation_ and the _carrier set_).
- List concatenation and an empty list

```python
>>> 42 + 0
42
>>> 42 * 1
42
>>> [1, 4, 3] + []
[1, 4, 3]
>>>
```

Given what we now know, we could rephrase the definition as "a monoid is a tuple of a carrier set, a binary operation, and an identity element that are closed and associative."

In [4]:
import operator as op
from functools import reduce
import typing as t

T = t.TypeVar("T")


class Monoid(t.Generic[T]):
    def __init__(self, operation, identity) -> None:
        self._op = operation
        self._ident: T = identity

    def fn(self, this: T, other: T) -> T:
        return self._op(this, other)


def foldr(monoid: Monoid[T], init_val: T, vals: t.Iterable[T]) -> T:
    return reduce(monoid.fn, vals, init_val)


Sum: Monoid[int] = Monoid(op.add, 0)


def test_identity() -> None:
    # Identity
    x = 42
    assert Sum.fn(42, 0) == x
    assert Sum.fn(0, 42) == x


def test_associativity() -> None:
    # Associativity
    Sum.fn(Sum.fn(7, 12), 42) == Sum.fn(7, Sum.fn(12, 42))


def test_right_fold() -> None:
    # Right fold
    assert foldr(Sum, 0, range(10)) == sum(range(10))

## References

- Weisstein, Eric W. "Monoid." From MathWorld--A Wolfram Web Resource. https://mathworld.wolfram.com/Monoid.html 
- https://en.wikipedia.org/wiki/Monoid
- https://wiki.haskell.org/Monoid